In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import statsmodels as sms
import statsmodels.formula.api as smf

import seaborn as sns # for data visualization
sns.set_style("whitegrid")

from dateutil.relativedelta import *
from pandas.tseries.offsets import *

pd.set_option('display.max_columns', None)

In [2]:
%%time

fundr = pd.read_csv('~/misp_data/fund_ratios_1970-2020.csv', index_col=0)
ibes = pd.read_csv('~/misp_data/ibes_raw_1976-2020.csv', index_col=0)
comp = pd.read_csv('~/misp_data/comp_1959-2019.csv', index_col=0)

/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


CPU times: user 1min 8s, sys: 6.89 s, total: 1min 15s
Wall time: 1min 15s


In [3]:
# import fund_ratio data since 1970

from dateutil.relativedelta import *
from pandas.tseries.offsets import *

fund_r = fundr[fundr['qdate'].notna()]
fund_r['date'] = pd.to_datetime((fund_r['qdate']))
fund_r['jdate']=fund_r['date']+MonthEnd(0)
fund_r['year'] = fund_r['jdate'].dt.year

/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [4]:
compvars = [ 'at',
                     'ni',
                     'epspi',
                     'revt',
                     'capx',
                     'naics',
                     'cogs',
                     'xsga',
                     'xrd',
                     'xad',
                     'ib',
                     'ebitda',
                     'ebit',
                     'nopi',
                     'pi',
                     'dvt',
                     'be',]

In [5]:
%%time

mdf = comp
mdf = mdf.sort_values(['permno', 'year'], ascending=[True, False])

eng_yr = 10
pred_var_yr = 5
pred_i = list(range(-pred_var_yr, 0))

for i in range(-pred_var_yr, eng_yr+1):
    mdf[f'ni_{i}'] = mdf.groupby('permno')['ni'].shift(i)
    print(i)
    if i in pred_i:
        for var in compvars:
            mdf[f'{var}_{i}'] = mdf.groupby('permno')[var].shift(i)

-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
CPU times: user 20.5 s, sys: 43.1 s, total: 1min 3s
Wall time: 1min 4s


In [6]:
%%time

for i in range(1, 6):
    print(i)
    for var in compvars: 
        mdf[f'{var}_yoy{i}'] = ((mdf[f'{var}'] - mdf[f'{var}_-{i}']) / mdf[f'{var}'])**(1/i)

1
2
3
4
5
CPU times: user 1.07 s, sys: 56.5 ms, total: 1.13 s
Wall time: 1.21 s


In [7]:
mdf['jdate'] = pd.to_datetime((mdf['jdate']), format='%Y-%m-%d')
mdf = mdf.merge(fund_r, how='left', on=['permno', 'jdate'])

In [8]:
mdf['year'] = mdf['year_y']

In [9]:
crsp = pd.read_csv('~/misp_data/crsp_1959-2019.csv', index_col=0)

/home/mma3/.conda/envs/mlval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
crsp['jdate'] = pd.to_datetime(crsp['jdate'])
mdf1 = mdf.merge(crsp, on=['jdate', 'permno'], how='left')

In [23]:
xvars = ['permno', 'ticker', 'jdate', 'jdate_crsp',
    'siccd',
 'sic2',
 'sic',
 'me',
 'at',
 'pstkl',
 'txditc',
 'pstkrv',
 'seq',
 'pstk',
 'ni',
 'epspi',
 'revt',
 'capx',
 'ajex',
 'naics',
 'sale',
 'cogs',
 'xsga',
 'xrd',
 'xad',
 'ib',
 'ebitda',
 'ebit',
 'nopi',
 'spi',
 'pi',
 'txp',
 'txfed',
 'txfo',
 'txt',
 'xint',
 'oancf',
 'dvt',
 'ob',
 'gdwlia',
 'gdwlip',
 'gwo',
 'rect',
 'act',
 'che',
 'ppegt',
 'invt',
 'aco',
 'intan',
 'ao',
 'ppent',
 'gdwl',
 'fatb',
 'fatl',
 'lct',
 'dlc',
 'dltt',
 'lt',
 'dm',
 'dcvt',
 'cshrc',
 'dcpstk',
 'ap',
 'lco',
 'lo',
 'drc',
 'drlt',
 'txdi',
 'ceq',
 'scstkc',
 'emp',
 'csho',
 'prcc_f',
 'mve_f',
 'am',
 'txdb',
 'dvc',
 'dvp',
 'dp',
 'dvpsx_f',
 'mib',
 'ivao',
 'ivst',
 'sstk',
 'prstkc',
 'dv',
 'dltis',
 'dltr',
 'dlcch',
 'oibdp',
 'dvpa',
 'tstkp',
 'oiadp',
 'xpp',
 'xacc',
 're',
 'ppenb',
 'ppenls',
 'capxv',
 'fopt',
 'wcap',
 'be',
 'at_-5',
 'revt_-5',
 'capx_-5',
 'ni_-5',
 'at_-4',
 'revt_-4',
 'capx_-4',
 'ni_-4',
 'at_-3',
 'revt_-3',
 'capx_-3',
 'ni_-3',
 'at_-2',
 'revt_-2',
 'capx_-2',
 'ni_-2',
 'at_-1',
 'revt_-1',
 'capx_-1',
 'ni_-1',
 'ni_yoy1',
 'revt_yoy1',
 'at_yoy1',
 'capx_yoy1',
 'ni_yoy3',
 'revt_yoy3',
 'at_yoy3',
 'capx_yoy3',
 'ni_yoy2',
 'revt_yoy2',
 'at_yoy2',
 'capx_yoy2',
 'CAPEI',
 'bm',
 'evm',
 'pe_op_basic',
 'pe_op_dil',
 'pe_exi',
 'pe_inc',
 'ps_y',
 'pcf',
 'dpr',
 'npm',
 'opmbd',
 'opmad',
 'gpm',
 'ptpm',
 'cfm',
 'roa',
 'roe',
 'roce',
 'efftax',
 'aftret_eq',
 'aftret_invcapx',
 'aftret_equity',
 'pretret_noa',
 'pretret_earnat',
 'GProf',
 'equity_invcap',
 'debt_invcap',
 'totdebt_invcap',
 'capital_ratio',
 'int_debt',
 'int_totdebt',
 'cash_lt',
 'invt_act',
 'rect_act',
 'debt_at',
 'debt_ebitda',
 'short_debt',
 'curr_debt',
 'lt_debt',
 'profit_lct',
 'ocf_lct',
 'cash_debt',
 'fcf_ocf',
 'lt_ppent',
 'dltt_be',
 'debt_assets',
 'debt_capital',
 'de_ratio',
 'intcov',
 'intcov_ratio',
 'cash_ratio',
 'quick_ratio',
 'curr_ratio',
 'cash_conversion',
 'inv_turn',
 'at_turn',
 'rect_turn',
 'pay_turn',
 'sale_invcap',
 'sale_equity',
 'sale_nwc',
 'rd_sale',
 'adv_sale',
 'staff_sale',
 'accrual',
 'ptb',
 'PEG_trailing',
 'PEG_1yrforward',
 'PEG_ltgforward',
  'ni_1','ni_2','ni_3', 'ni_4','ni_5',
        'ni_6','ni_7','ni_8', 'ni_9','ni_10']
mdf1 = mdf1[xvars]

In [24]:
mdf1.head()

,permno,ticker,jdate,jdate_crsp,siccd,sic2,sic,me,at,pstkl,txditc,pstkrv,seq,pstk,ni,epspi,revt,capx,ajex,naics,sale,cogs,xsga,xrd,xad,ib,ebitda,ebit,nopi,spi,pi,txp,txfed,txfo,txt,xint,oancf,dvt,ob,gdwlia,gdwlip,gwo,rect,act,che,ppegt,invt,aco,intan,ao,ppent,gdwl,fatb,fatl,lct,dlc,dltt,lt,dm,dcvt,cshrc,dcpstk,ap,lco,lo,drc,drlt,txdi,ceq,scstkc,emp,csho,prcc_f,mve_f,am,txdb,dvc,dvp,dp,dvpsx_f,mib,ivao,ivst,sstk,prstkc,dv,dltis,dltr,dlcch,oibdp,dvpa,tstkp,oiadp,xpp,xacc,re,ppenb,ppenls,capxv,fopt,wcap,be,at_-5,revt_-5,capx_-5,ni_-5,at_-4,revt_-4,capx_-4,ni_-4,at_-3,revt_-3,capx_-3,ni_-3,at_-2,revt_-2,capx_-2,ni_-2,at_-1,revt_-1,capx_-1,ni_-1,ni_yoy1,revt_yoy1,at_yoy1,capx_yoy1,ni_yoy3,revt_yoy3,at_yoy3,capx_yoy3,ni_yoy2,revt_yoy2,at_yoy2,capx_yoy2,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps_y,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,GProf,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,PEG_1yrforward,PEG_ltgforward,ni_1,ni_2,ni_3,ni_4,ni_5,ni_6,ni_7,ni_8,ni_9,ni_10
0,10000,OMFGA,1986-10-31,1986-10-31,3990.0,3942,3942,3002.34375,2.115,0.0,0.0,0.0,0.418,0.0,-0.730,-0.20,1.026,0.240,1.0,339931.0,1.026,0.511,1.100,0.039,0.113,-0.730,-0.585,-0.686,0.024,0.0,-0.730,0.0,0.0,0.0,0.0,0.068,NaN,0.0,NaN,NaN,NaN,NaN,0.700,1.630,0.348,0.328,0.512,0.070,0.252,0.044,0.189,NaN,NaN,NaN,1.639,0.968,0.058,1.697,NaN,0.0,0.0,0.0,0.410,0.261,0.0,NaN,NaN,0.0,0.418,NaN,0.053,3.843,0.75,2.88225,0.050,0.0,0.0,0.0,0.101,0.0,0.0,0.0,0.0,1.743,0.0,0.0,0.057,0.0,-0.372,-0.585,0.0,0.0,-0.686,0.070,NaN,-1.599,NaN,NaN,0.240,-0.507,-0.009,0.418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.72,0.354,0.026,-0.511,0.3,0.654971,0.659574,0.891667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145,-6.681,NaN,NaN,-2.345,-2.345,1.931,-1.469,NaN,-0.712,-0.57,-0.669,0.502,-0.712,-0.567,-0.413,NaN,-0.808,NaN,-19.467,-0.102,-19.467,3.049,-0.609,0.243,0.878,0.122,2.155,0.122,2.345,0.084,0.205,0.314,0.429,0.485,-1.754,0.943,0.966,0.034,-0.357,-0.823,-0.795,NaN,8.979,0.139,0.802,0.775,4.06,-9.735,-10.088,0.212,0.682,0.995,148.069,1.237,0.724,2.842,1.839,2.155,2.455,NaN,0.038,0.11,0.0,-0.437,4.741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,OMFGA,1986-10-31,1986-10-31,3990.0,3942,3942,3002.34375,2.115,0.0,0.0,0.0,0.418,0.0,-0.730,-0.20,1.026,0.240,1.0,339931.0,1.026,0.511,1.100,0.039,0.113,-0.730,-0.585,-0.686,0.024,0.0,-0.730,0.0,0.0,0.0,0.0,0.068,NaN,0.0,NaN,NaN,NaN,NaN,0.700,1.630,0.348,0.328,0.512,0.070,0.252,0.044,0.189,NaN,NaN,NaN,1.639,0.968,0.058,1.697,NaN,0.0,0.0,0.0,0.410,0.261,0.0,NaN,NaN,0.0,0.418,NaN,0.053,3.843,0.75,2.88225,0.050,0.0,0.0,0.0,0.101,0.0,0.0,0.0,0.0,1.743,0.0,0.0,0.057,0.0,-0.372,-0.585,0.0,0.0,-0.686,0.070,NaN,-1.599,NaN,NaN,0.240,-0.507,-0.009,0.418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.72,0.354,0.026,-0.511,0.3,0.654971,0.659574,0.891667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145,-6.681,NaN,NaN,-1.875,-1.875,1.541,-1.172,NaN,-0.712,-0.57,-0.669,0.502,-0.712,-0.567,-0.413,NaN,-0.808,NaN,-19.467,-0.102,-19.467,3.049,-0.609,0.243,0.878,0.122,2.155,0.122,2.345,0.084,0.205,0.314,0.429,0.485,-1.754,0.943,0.966,0.034,-0.357,-0.823,-0.795,NaN,8.979,0.139,0.802,0.775,4.06,-9.735,-10.088,0.212,0.682,0.995,148.069,1.237,0.724,2.842,1.839,2.155,2.455,NaN,0.038,0.11,0.0,-0.437,3.784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000,OMFGA,1986-10-31,1986-10-31,3990.0,3942,3942,3002.34375,2.115,0.0,0.0,0.0,0.418,0.0,-0.730,-0.20,1.026,0.240,1.0,339931.0,1.026,0.511,1.100,0.039,0.113,-0.730,-0.585,-0.686,0.024,0.0,-0.730,0.0,0.0,0.0,0.

In [25]:
mdf1 = mdf1.drop_duplicates(subset=['permno', 'jdate'], keep='last')

In [26]:
mdf1[mdf1.ticker=='TSLA']

,permno,ticker,jdate,jdate_crsp,siccd,sic2,sic,me,at,pstkl,txditc,pstkrv,seq,pstk,ni,epspi,revt,capx,ajex,naics,sale,cogs,xsga,xrd,xad,ib,ebitda,ebit,nopi,spi,pi,txp,txfed,txfo,txt,xint,oancf,dvt,ob,gdwlia,gdwlip,gwo,rect,act,che,ppegt,invt,aco,intan,ao,ppent,gdwl,fatb,fatl,lct,dlc,dltt,lt,dm,dcvt,cshrc,dcpstk,ap,lco,lo,drc,drlt,txdi,ceq,scstkc,emp,csho,prcc_f,mve_f,am,txdb,dvc,dvp,dp,dvpsx_f,mib,ivao,ivst,sstk,prstkc,dv,dltis,dltr,dlcch,oibdp,dvpa,tstkp,oiadp,xpp,xacc,re,ppenb,ppenls,capxv,fopt,wcap,be,at_-5,revt_-5,capx_-5,ni_-5,at_-4,revt_-4,capx_-4,ni_-4,at_-3,revt_-3,capx_-3,ni_-3,at_-2,revt_-2,capx_-2,ni_-2,at_-1,revt_-1,capx_-1,ni_-1,ni_yoy1,revt_yoy1,at_yoy1,capx_yoy1,ni_yoy3,revt_yoy3,at_yoy3,capx_yoy3,ni_yoy2,revt_yoy2,at_yoy2,capx_yoy2,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps_y,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,GProf,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,PEG_1yrforward,PEG_ltgforward,ni_1,ni_2,ni_3,ni_4,ni_5,ni_6,ni_7,ni_8,ni_9,ni_10
776760,93436,TSLA,2019-12-31,2019-12-31,9999.0,3711,3711,7.540189e+07,34309.000,0.0,0.000,0.0,6618.000,0.000,-862.000,-4.92,24578.000,1432.000,5.0,336111.0,24578.000,18402.000,3989.000,1390.000,27.0,-862.000,2187.000,80.000,120.000,-149.000,-665.000,611.000,0.000,86.000,110.000,716.000,2405.000,0.0,NaN,NaN,NaN,NaN,1324.000,12103.000,6514.000,25062.000,3552.000,713.000,537.000,1077.000,20199.000,198.000,NaN,NaN,10667.000,2070.000,12627.000,26199.000,5950.000,3589.000,NaN,3589.000,3771.000,4215.000,2905.000,1163.000,1207.000,19.000,6618.000,NaN,48.016,181.000,418.330,75717.730000,44.0,0.000,0.0,0.0,2107.000,0.0,643.000,393.000,246.000,1111.000,0.0,0.0,10669.000,9871.000,NaN,2187.000,0.0,0.0,80.000,NaN,1534.000,-6119.000,NaN,NaN,1432.000,NaN,1436.000,6618.000,5849.251,3198.356,969.885,-294.040,8092.460,4046.025,1634.850,-888.663,22664.076,7000.132,1440.471,-674.914,28655.372,11758.751,4081.354,-1961.400,29739.614,21461.268,2319.516,-976.091,-0.132356,0.126810,0.133183,-0.619774,0.600959,0.894279,0.697551,NaN,NaN,0.722201,0.405938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
776763,93436,TSLA,2018-12-31,2018-12-31,9999.0,3711,3711,5.744194e+07,29739.614,0.0,0.000,0.0,4923.243,0.000,-976.091,-5.72,21461.268,2319.516,5.0,336111.0,21461.268,15531.461,4294.861,1473.634,70.0,-976.091,1634.946,-252.840,101.299,-135.233,-1004.745,348.663,-0.901,23.622,57.837,717.971,2097.802,0.0,NaN,NaN,NaN,NaN,949.022,8306.308,3878.169,23343.447,3113.446,365.671,350.651,969.876,19691.231,68.159,NaN,NaN,9992.136,2629.460,9454.055,23426.010,NaN,2206.621,NaN,2206.621,3404.451,3609.562,3979.819,630.292,990.873,32.324,4923.243,NaN,48.817,172.603,332.800,57442.278400,66.0,0.000,0.0,0.0,1887.786,0.0,555.964,421.548,192.551,295.722,0.0,0.0,6176.173,6087.029,NaN,1634.946,0.0,0.0,-252.840,NaN,1121.670,-5326.050,NaN,NaN,2319.516,NaN,-1685.828,4923.243,2416.930,2013.496,264.224,-74.014,5849.251,3198.356,969.885,-294.040,8092.460,4046.025,1634.850,-888.663,22664.076,7000.132,1440.471,-674.914,28655.372,11758.751,4081.354,-1961.400,-1.009444,0.452094,0.036458,-0.759571,0.447425,0.932735,0.899543,0.665826,0.555477,0.820868,0.487767,0.615612,-43.237,0.086,42.865,-46.802,-46.802,-41.729,-41.729,1.932,19.766,NaN,-0.045,0.076,-0.012,0.276,-0.047,0.046,0.056,-0.213,-0.016,NaN

In [27]:
mdf1['year'] = mdf1['jdate'].dt.year.astype(int)
mdf1.to_csv('~/misp_data/lagged_comp_fundr_for_val_1976-2019.csv')

In [28]:
mdf1.shape[1]/2

107.5